# Assignment 2

In this assignment, we will focus on regular expressions and byte-pair encoding. Assignment 2 is worth 10% toward your final grade.

## [50 marks] Regular expression

1. [30marks] Write python code with regular expressions to clean the html webpage.

  Input: https://slpcourse.github.io/materials/html_page.txt

  Reference Output: https://slpcourse.github.io/materials/reference.txt

2. [20 marks] Based on the output from 1, extract the lines with lecture time, tutorial time and office hours. Your need to use regular expressions.


Note: We expect the regular expressions to be just enough for the task. If there are extra non-used regular expressions, we will deduct scores based the lines of non-used regular expressions. Each line that contains non-used regular expressions is worth 5 marks. Each uncleaned html tag is worth 2 points. Each unnecessary whitespace is worth 1 point.

In [19]:
## question 1.
import re
import requests

url = "https://slpcourse.github.io/materials/html_page.txt"

response = requests.get(url)
html_content = response.text

def clean_html(raw_html):
  # Clean html tags
  pattern = r'<a\s+[^>]*>(.*?)</a>' # URL pattern in HTML webpage
  withoutURL_html = re.sub(pattern, r'\1', raw_html, flags=re.DOTALL) # delete URL

  # Remove all other HTML tags
  withoutTags_text = re.sub('<.*?>', '', withoutURL_html)

  # Clean unnecessary white space
  withoutIndent_text = re.sub('\t+', '', withoutTags_text) # delete indent spaces
  withoutBlanklines_text = re.sub('\n+', '\n', withoutIndent_text) # delete blank new lines
  cleantext = re.sub('\n ', '\n', withoutBlanklines_text) # deleting white space to left align all text

  return cleantext.strip()

# Clean the HTML content
cleaned_text = clean_html(html_content)

# Display the cleaned text for verification
print(cleaned_text)

CSC3160 Fundamentals of Speech and Language Processing
MDS6002 Natural Language Processing
Spring 2023
The difference between speech and language processing and other data processing is the use of knowledge
of language. In this course, we will study how to describe, process and compute different levels of
language knowledge including Phonetics and Phonology, Morphology, Syntax, Semantics, and how the
language knowledge is used in speech and language applications such as named entities recognition,
information extraction, question answering, speech recognition, and speech synthesis.
Teaching team
Instructor  Zhizheng Wu
TA  Xi Chen
TA Xueyao Zhang
Poster Session
A final project poster session is planned by the end of the course (tentatively May 20th 2023). This
is to provide students the opportunities to connect with speech and language research/industry
community.
Anyone from the CUHK-Shenzhen and speech and language technology community are welcome to join. More
details will be provid

In [20]:
## question 2.
# extract instructor's and TA's names.
Instructor_name=re.compile(r'Instructor.*').findall(cleaned_text)
Instructor_name=Instructor_name[0].split('Instructor')[1].strip()
TA_names=re.compile(r'TA.*').findall(cleaned_text)
TA_names1=TA_names[0].split('TA')[1].strip()
TA_names2=TA_names[1].split('TA')[1].strip()

# Get lecture time
lecture_time = re.compile(r'Lectures[^.!?]*').findall(cleaned_text)[0]
print("Lecture time: ")
print(lecture_time)

# Get tutorial time
tutorial_time = re.compile(r'[^.!?]*tutorial[^.!?]*').findall(cleaned_text)
print("\nTutorial time: ")
print(tutorial_time)
print("From the text above, we know there is no offline tutorial.\n")

# use 'Office hours', name of instructor, and names of TAs to locate office hours.
office_hours=re.compile('Office hours.*%s.*%s.*%s[^\n]*'%(Instructor_name,TA_names1,TA_names2), re.MULTILINE|re.DOTALL).findall(cleaned_text)[0]
print(office_hours)

Lecture time: 
Lectures: are on Tuesday/Thursday 4:00PM - 5:20PM in TB103

Tutorial time: 
[' If you are not familiar with LaTex, please learn from the tutorial in advance']
From the text above, we know there is no offline tutorial.

Office hours 
Zhizheng Wu: Thu 2:30-3:30 PM. Daoyuan Building 321b
Xi Chen: Wed 7-9PM. SDS Research Lab (4th Floor, Zhi Xin Building) Seat No.100
Xueyao Zhang: Wed 7-9PM. SDS Research Lab (4th Floor, Zhi Xin Building) Seat No.78


## [50 marks] Byte-pair encoding

In this assignment, the task is to implement a Byte-Pair Encoding (BPE) algorithm to learn subword tokens.

Here is a vocabulary with frequency,
```
2 oneumonoultramicroscopicsilicovolcanoconiosis
5 hippopotomonstrosesquippedaliophobia
4 supercalifragilisticexpialidocious
1 pseudopseudohypoparathyroidism
6 floccinaucinihilipilification
2 antidisestablishmentarianism
5 honorificabilitudinitatibus
```
The first column represents the occurency frequency, and the second column represents the words.

In the implementation, k is set to be 5.

In [22]:
from collections import defaultdict
import re

# Count numbers of adjacent tokens
def get_stats(vocab):
  pairs = defaultdict(int)
  for word, freq in vocab.items():
    symbols = word.split()
    for i in range(len(symbols) - 1):
      pairs[symbols[i], symbols[i + 1]] += freq
  return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
      w_out = p.sub(''.join(pair), word)
      v_out[w_out] = v_in[word]
    return v_out

def bpe(vocab, K):
  for i in range(K):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    print('The most frequent pair of adjacent tokens in round '+str(i)+': ',best)
    vocab = merge_vocab(best, vocab)
  return vocab

# Input vocabulary with frequency
vocab = {"oneumonoultramicroscopicsilicovolcanoconiosis": 2,
         "hippopotomonstrosesquippedaliophobia": 5,
         "supercalifragilisticexpialidocious": 4,
         "pseudopseudohypoparathyroidism": 1,
         "floccinaucinihilipilification": 6,
         "antidisestablishmentarianism": 2,
         "honorificabilitudinitatibus": 5}

# Convert each word in the vocabulary to a space-separated string of characters
vocab = {(' '.join(word) + ' _'): freq for word, freq in vocab.items()}

# Number of merges
K = 5

# Perform BPE
bpe_vocab = bpe(vocab, K)

# Display the modified vocabulary
print('The vocabulary after merging 5 times using BPE algorithm: ')
bpe_vocab

The most frequent pair of adjacent tokens in round 0:  ('l', 'i')
The most frequent pair of adjacent tokens in round 1:  ('i', 'li')
The most frequent pair of adjacent tokens in round 2:  ('o', 'n')
The most frequent pair of adjacent tokens in round 3:  ('i', 'c')
The most frequent pair of adjacent tokens in round 4:  ('i', 'n')
The vocabulary after merging 5 times using BPE algorithm: 


{'on e u m on o u l t r a m ic r o s c o p ic s ili c o v o l c a n o c on i o s i s _': 2,
 'h i p p o p o t o m on s t r o s e s q u i p p e d a li o p h o b i a _': 5,
 's u p e r c a li f r a g ili s t ic e x p i a li d o c i o u s _': 4,
 'p s e u d o p s e u d o h y p o p a r a t h y r o i d i s m _': 1,
 'f l o c c in a u c in i h ili p ili f ic a t i on _': 6,
 'a n t i d i s e s t a b li s h m e n t a r i a n i s m _': 2,
 'h on o r i f ic a b ili t u d in i t a t i b u s _': 5}